<div>
    <img src="https://www.ient.rwth-aachen.de/cms/uploads/images/rwth_ient_logo@2x.png" style="float: right;height: 5em;">
</div>

In [ ]:
# Copyright 2019 Institut für Nachrichtentechnik, RWTH Aachen University

# Default Imports
import sys
import os
import numpy as np

# Path inserts
sys.path.insert(0,'./Bilder')
sys.path.insert(0,'../ient_python')

# Imports for Widgets
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact

# Imports for combining languages
import ctypes
from cython import *
from ctypes import *
from ctypes import cdll
from numpy.ctypeslib import ndpointer
from numpy.ctypeslib import as_array

# Imports for image processing and displaying
from skimage.morphology import label
from skimage.io import imread
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.morphology import closing, square
from skimage.util import img_as_int
from skimage.measure import regionprops, centroid, moments_central, moments_coords_central
from skimage.draw import circle_perimeter
from matplotlib.pyplot import imshow
import matplotlib.image
from IPython.display import *
import matplotlib.patches as mpatches

# Remove .so file if exists
if os.path.exists('mass_calculation.so'):
    os.remove('mass_calculation.so')

# Suppress warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Suppress figure warning
plt.close('all')
plt.rcParams.update({'figure.max_open_warning': 0})

## Teil 2: Erweiterte Merkmalsbestimmung und Python-C++ Kombination
In diesem Teil des Versuchs geht es um zwei Dinge:
 - Eine weitere Art an Beschreibungsmöglichkeiten von Objekten in Bildern kennenzulernen
 - Die Möglichkeit verschiedene Programmiersprachen zu kombinieren

Für nähere Informationen bezüglich der verwendeten Funktionen, schauen Sie hier:

__[Befehlsreferenz](Befehlsreferenz_V6.ipynb)__

Eine weitere Erklärung zur Schnittstellenprogrammierung und zu Momenten finden Sie in nachfolgendem Video.

In [ ]:
from IPython.display import Video
Video("https://www.ient.rwth-aachen.de/cms/uploads/public/video/PTI-Videos/V6_T2_v3.mp4", width=480, height=270 )

<div class="alert rwth-topic"> 
    
## Aufgabe 1 - Ermittlung des Massenschwerpunktes 
Ermitteln Sie zunächst den Massenschwerpunkt, um die geometrische Lage der Objekte im Bild zu charakterisieren.
    
</div>

<div class="alert rwth-subtopic">
    
### Aufgabe 1.1: C-Programm erstellen 
Bevor Sie weiter in Jupyter Notebook arbeiten, öffnen Sie bitte die Datei *mass_calculation.c* und vervollständigen dort die Funktion zur Berechnung des Massenschwerpunktes von Objekten in Bildern. Nutzen Sie die Pseudocode-Version aus den Vorbereitungsaufgaben zur Hilfe.
    
Wenn Sie glauben, dass ihr Programm funktioniert, führen sie den nächsten Block aus. Dieser kompiliert Ihre C-Datei. Bei Fehlermeldungen überprüfen Sie ihr Programm erneut, bis keine Fehler mehr auftreten.  Die Ausgabe der Zelle beinhaltet dann nur noch '[ ]'     
</div>

In [ ]:
#compile the c-file via gcc-compiler into an shared object(.so file) with fPIC gcc-option in order to have relative adresses
!!gcc -shared -Wl,-soname,mass_calculation \
    -o mass_calculation.so \
    -fPIC mass_calculation.c

<div class="alert rwth-subtopic">
    
### Funktion verwenden
Glückwunsch, jetzt kann Ihre Funktion verwendet werden. Es soll nun noch kontrolliert werden, ob die errechneten Ergebnisse richtig sind. Dazu werden im folgenden auch die Massenschwerpunkte durch eine vorimplementierte Pythonfunktion berechnet und mit den Ergebnissen Ihrer Funktion verglichen.
Sollten die Ergebniss nicht übereinstimmen, müssen Sie ihre Funktion nocheinmal überarbeiten.
    
Die nachfolgende Zelle ruft Ihre Funktion auf und zeigt Ihnen eine grafische Veranschaulichung Ihrer Ergebnisse und der Ergebnisse durch die vorimplementierte Funktion.

</div>  

In [ ]:
# Pointer definieren, der die Adresse vom Typ intpointer (adress) speichert
doublepp = ndpointer(dtype=np.intp) 

# Das im obigen Block erstellte .so File mittles ctypes Befehl cddll
#Load the created .so-file via ctypes into python-variable (and create a .py-file)
dll = cdll.LoadLibrary('./mass_calculation.so')

#rename function for easy use and access
center = dll.center
central = dll.central

###define interface
center.restype = None
center.argtypes = [doublepp,c_int ,doublepp]

central.restype = None
central.argtypes = [doublepp, c_int, doublepp, doublepp, c_int, c_int]

#Bild für spätere Übergabe vorbereiten (Laden, Graustufenbild, Binarisieren, invertieren, Labeln)
img_gray = imread('Bilder/Buchstaben_Teil2.bmp')
img_bin = img_gray > threshold_otsu(img_gray)
img_inv = ~img_bin
img_labeled = label(img_inv)

# labeled image as double for pointer-pointer
img_labeled_db = img_labeled.astype(np.double)

# Vektor, in welchen die Funktion die Ergebnisse schreibt
results = np.zeros((14), dtype = 'double')

#2 Pointer-Pointer-Arrays als Hilfsmittel 
res_tmp = (results.__array_interface__['data'][0]+np.arange(results.shape[0])*results.strides[0])
img_labeled_pp = (img_labeled_db.__array_interface__['data'][0]+np.arange(img_labeled_db.shape[0])*img_labeled_db.strides[0])


# Ausgabe der wichtigsten Eigenschaften der Matrix, zur Übergabe an die Funktion
_,labels_num = label(img_inv, return_num=True)
_, dimm = img_labeled.shape
dimn,_ = img_labeled.shape
# Aufruf Ihrer eigenen Funktion
center(img_labeled_pp, labels_num, res_tmp)

# Gebietseigenschaften berechnen
regions = regionprops(img_labeled)

# Schwerpunkte jedes Gebiets berechnen und speichern
sk_centers = np.zeros((7,2))
for i in range(7): # da 7 Gebiete wie oben angegeben
    sk_centers[i,0], sk_centers[i,1] = regions[i].centroid # Koordinaten des Schwerpunkts in Form: (Zeile, Spalte)
    
    
    
#Arrangiere Ergebnisse neu für schönere Ausgabe 
coord = results.reshape((7, 2))
coord_tmp = np.array_str(coord)
coord_head = "  Selbst errechnete Lösung: \n" + coord_tmp 

sk_coords = sk_centers.reshape((7, 2))
sk_coord_tmp = np.array_str(sk_coords)
sk_coord_head = " Von Skimage errechnete Lösung: \n" + sk_coord_tmp 


# Visualisierung der Ergebnisse
def circles(sk_centers, coord):
    img_labeled_cp = np.copy(img_labeled)  
    img_labeled_cp_cp = np.copy(img_labeled)

    # Hier kommt Ihre Lösung zu Unterpunkt 1.5:
    fig, ax = plt.subplots(1, 2, figsize=(12, 8))
    ax[0].imshow(img_labeled_cp, cmap= 'gray');
    ax[0].set_title('Visualisierte Schwerpunkte Skimage:')
    ax[1].imshow(img_labeled_cp_cp, cmap= 'gray');
    ax[1].set_title('Visualisierte Schwerpunkte eigener Code:')
    ax[0].axis('off')
    ax[1].axis('off')

    for i in range(7):
        circle = mpatches.Circle((int(sk_centers[i][1]),int(sk_centers[i][0])) ,radius = 5)
        circle.set_color('red')
        ax[0].add_patch(circle)

    for i in range(7):
        circle = mpatches.Circle((int(coord[i][1]),int(coord[i][0])) ,radius = 5)
        ax[1].add_patch(circle)

    plt.show()
    
circles(sk_centers, coord)

# Ausgabe der Ergebnisse
print(coord_head,'\n\n', sk_coord_head)

# >>Weiter zu  [__Teil 3__](Teil_3.ipynb)